## 12: Hierarchical 2SFCA for Equitable Access to Care

**Goal:** To model healthcare accessibility by accounting for both resource competition (supply vs. demand) and the need to match patients with an appropriate level of care. This moves beyond simple travel time to a more nuanced measure of service equity.

**Methodology:**
We will implement the **Hierarchical Two-Step Floating Catchment Area (H-2SFCA)** method.
1.  **Define Service Tiers:** Assign capabilities to our facility hierarchy (GP, Community, Acute).
2.  **Step 1 (Supply-Side):** For each facility, calculate its provider-to-population ratio within its service area.
3.  **Step 2 (Demand-Side):** For each residential area (LSOA), sum the ratios of all *appropriate* facilities it can reach.
4.  **Visualize Tiered Access:** Create separate maps for Primary, Community, and Acute care accessibility to identify different types of healthcare deserts.

### 1. Setup and Library Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point, box
import contextily as cx

### 2. Load and Prepare Data

We need a facility hierarchy, demand points (LSOAs with population), and a street network.

In [ ]:
# --- 1. Facility Data with Hierarchy and Capacity ---
facilities_data = {
    'Royal Devon and Exeter (Acute)': [-3.503, 50.713, 3, 50], # Name: [lon, lat, tier, capacity (e.g., # of specialists)]
    'Exeter Community Hospital': [-3.518, 50.718, 2, 20],
    'Heavitree Hospital (CDC)': [-3.495, 50.720, 2, 25],
    'St. Thomas Medical Group (GP)': [-3.542, 50.717, 1, 5],
    'Pinhoe & Broadclyst Medical (GP)': [-3.475, 50.741, 1, 6],
    'Ide Lane Surgery (GP)': [-3.545, 50.705, 1, 4],
    'Topsham Surgery (GP)': [-3.468, 50.682, 1, 3]
}
df = pd.DataFrame.from_dict(facilities_data, orient='index', columns=['lon', 'lat', 'tier', 'capacity'])
facilities_gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)

# --- 2. Demand Data (LSOAs with Population) ---
# Using placeholder LSOAs for Exeter as in previous notebooks
xmin, ymin, xmax, ymax = -3.58, 50.68, -3.42, 50.78 # Bounding box for Exeter
grid_cells = []
for i, x in enumerate(np.linspace(xmin, xmax, 15)):
    for j, y in enumerate(np.linspace(ymin, ymax, 15)):
        grid_cells.append(box(x, y, x + (xmax-xmin)/15, y + (ymax-ymin)/15))
lsoa_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:4326")
lsoa_gdf['LSOA_ID'] = range(len(lsoa_gdf))
np.random.seed(42)
lsoa_gdf['population'] = np.random.randint(1500, 2500, size=len(lsoa_gdf))
demand_points = lsoa_gdf.copy()
demand_points['geometry'] = demand_points.centroid

# --- 3. Street Network ---
G = ox.graph_from_place("Exeter, England", network_type='drive')

### 3. Implement the Hierarchical 2SFCA Method

We'll create a function to run the 2SFCA for a given service tier and travel time.

In [ ]:
def run_2sfca(facilities, demand, network, travel_time_min, service_tier):
    """Runs the 2SFCA algorithm for a specific service tier."""
    # --- Step 1: Calculate Provider-to-Population Ratios (PPR) ---
    ppr_list = []
    
    # Consider only facilities that can provide *at least* the required service tier
    relevant_facilities = facilities[facilities['tier'] >= service_tier]
    
    for idx, facility in relevant_facilities.iterrows():
        # Create catchment area
        center_node = ox.nearest_nodes(network, facility.geometry.x, facility.geometry.y)
        isochrone = ox.isochrone_polygons(network, center_node, trip_times=[travel_time_min], edge_attack=True)
        
        # Find demand points within catchment
        points_in_iso = demand[demand.within(isochrone.iloc[0].geometry)]
        population_in_catchment = points_in_iso['population'].sum()
        
        # Calculate PPR
        if population_in_catchment > 0:
            ppr = facility['capacity'] / population_in_catchment
        else:
            ppr = 0
        ppr_list.append(ppr)
    
    relevant_facilities['ppr'] = ppr_list
    
    # --- Step 2: Sum Ratios for Demand Points ---
    accessibility_scores = []
    for idx, demand_point in demand.iterrows():
        # Create catchment area for the demand point
        center_node = ox.nearest_nodes(network, demand_point.geometry.x, demand_point.geometry.y)
        isochrone = ox.isochrone_polygons(network, center_node, trip_times=[travel_time_min], edge_attack=True)
        
        # Find facilities within this catchment
        facilities_in_iso = relevant_facilities[relevant_facilities.within(isochrone.iloc[0].geometry)]
        
        # Sum the PPRs of reachable facilities
        accessibility_score = facilities_in_iso['ppr'].sum()
        accessibility_scores.append(accessibility_score)
        
    return accessibility_scores

# Run for each tier
lsoa_gdf['access_primary'] = run_2sfca(facilities_gdf, demand_points, G, 10, service_tier=1) # 10-min drive for GP
lsoa_gdf['access_community'] = run_2sfca(facilities_gdf, demand_points, G, 20, service_tier=2) # 20-min drive for Community
lsoa_gdf['access_acute'] = run_2sfca(facilities_gdf, demand_points, G, 30, service_tier=3) # 30-min drive for Acute

### 4. Visualize Tiered Accessibility

We create a separate choropleth map for each care tier. This allows us to see how different types of healthcare access vary across the city.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(24, 8), sharex=True, sharey=True)
lsoa_plot = lsoa_gdf.to_crs(epsg=3857)

# Map 1: Primary Care (GP) Accessibility
lsoa_plot.plot(column='access_primary', cmap='viridis', linewidth=0.5, ax=axes[0], edgecolor='0.8', legend=True,
               legend_kwds={'label': "Primary Care Score (GP)", 'orientation': "horizontal"})
axes[0].set_title('A) Primary Care Accessibility')
cx.add_basemap(axes[0], crs=lsoa_plot.crs.to_string(), source=cx.providers.CartoDB.Positron)
axes[0].set_axis_off()

# Map 2: Community Care Accessibility
lsoa_plot.plot(column='access_community', cmap='plasma', linewidth=0.5, ax=axes[1], edgecolor='0.8', legend=True,
               legend_kwds={'label': "Community Care Score", 'orientation': "horizontal"})
axes[1].set_title('B) Community Care Accessibility')
cx.add_basemap(axes[1], crs=lsoa_plot.crs.to_string(), source=cx.providers.CartoDB.Positron)
axes[1].set_axis_off()

# Map 3: Acute Care (Hospital) Accessibility
lsoa_plot.plot(column='access_acute', cmap='magma', linewidth=0.5, ax=axes[2], edgecolor='0.8', legend=True,
               legend_kwds={'label': "Acute Care Score (Hospital)", 'orientation': "horizontal"})
axes[2].set_title('C) Acute Care Accessibility')
cx.add_basemap(axes[2], crs=lsoa_plot.crs.to_string(), source=cx.providers.CartoDB.Positron)
axes[2].set_axis_off()

plt.tight_layout()
plt.show()

### 5. Analysis and Conclusion

The 2SFCA method provides a much richer understanding of accessibility than travel time alone.

- **Reveals Hidden Disparities:** An area might appear to have good access because it's close to a facility (low travel time), but if that facility is overwhelmed with demand from a large population, the actual accessibility is low. The 2SFCA score captures this.
- **Highlights Different 'Deserts':** The maps clearly show that an area can be a 'desert' for one type of care but not another. For example, a suburban area might have excellent access to GP practices (Map A) but very poor access to specialized acute care (Map C).
- **Informs Systemic Planning:** This analysis is vital for health service planners. It helps identify not just where to put new facilities, but *what kind* of facilities are needed most in specific areas to ensure equitable access across all tiers of care.

### 6. References and Further Reading

- **Luo, W., & Wang, F. (2003).** *Measures of spatial accessibility to health care in a GIS environment: synthesis and a case study in the Chicago region*. Environment and Planning B: Planning and Design, 30(6), 865-884. One of the foundational papers on the 2SFCA method.
- **Zhao, Y., & Zhou, Y. (2024).** *Isochrone-Based Accessibility Analysis of Pre-Hospital Emergency Medical Facilities*. ISPRS Int. J. Geo-Inf. The concept of assessing equity from the demand side by considering resource competition is central to this paper and the 2SFCA method.
- **Hierarchical Models:** The idea of modeling different tiers of service is common in healthcare and retail analysis. It recognizes that services are not uniform and that higher-order services are typically less numerous and have larger catchment areas.